In [1]:
from astropy.io import fits
from astropy.visualization import (ImageNormalize, MinMaxInterval, ZScaleInterval, LinearStretch)
from astropy.visualization import make_lupton_rgb  # Optional for color
from PIL import Image 
from io import BytesIO
import matplotlib.pyplot as plt
# import numpy as np
import cv2
import cupy as np
from skimage.color import rgb2hsv

b_channel_hdul = fits.open('assets/h_m51_b_s05_drz_sci.fits')
h_channel_hdul = fits.open('assets/h_m51_h_s05_drz_sci.fits')
v_channel_hdul = fits.open('assets/h_m51_v_s05_drz_sci.fits')

In [2]:
b_channel = b_channel_hdul[0].data
h_channel = h_channel_hdul[0].data
v_channel = v_channel_hdul[0].data
b_channel

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(12200, 8600), dtype='>f4')

In [3]:
b_channel = np.array(b_channel)
h_channel = np.array(h_channel)
v_channel = np.array(v_channel)
b_channel.max()

CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
b_ch_normalised = ImageNormalize(b_channel, interval=ZScaleInterval(), stretch=LinearStretch())
h_ch_normalised = ImageNormalize(h_channel, interval=ZScaleInterval(), stretch=LinearStretch())
v_ch_normalised = ImageNormalize(v_channel, interval=ZScaleInterval(), stretch=LinearStretch())

In [ ]:
rgb_image = np.stack([b_ch_normalised(b_channel), h_ch_normalised(h_channel), v_ch_normalised(v_channel)], axis=-1)
intensity_image = np.average(rgb_image, axis=-1)
hsv_image = rgb2hsv(rgb_image)

/home/trbf/Code/projects/NASA-IGZOOMERS/photo-zoomer/zoomervenv/lib/python3.12/site-packages/skimage/color/colorconv.py:316: RuntimeWarning: divide by zero encountered in divide
  out_s = delta / out_v


In [ ]:
# norm = ImageNormalize(np_img, interval=ZScaleInterval(), stretch=LinearStretch())
# plt.imshow(np_img, cmap='gray', origin='lower', norm=norm)
# plt.colorbar()
# plt.title('FITS Image (Z-Scale)')
# plt.show()

In [ ]:
hsv_image.shape

(12200, 8600, 3)

In [ ]:
plt.imshow('ree', hsv_image)
plt.axis('off')
plt.show()

In [ ]:
def select_zone(np_image: np.array, x: int, y: int, zoom_level: int, w_resolution: int = 1920, h_resolution: int = 1080):
    # returns required image section (x, y, x+1920*zoom_level)
    w_ct = np_image.shape[0] / w_resolution
    h_ct = np_image.shape[1] / h_resolution

    sectioned_array = np_image[x : x + w_ct * zoom_level, y : y + h_ct * zoom_level]
    
    return sectioned_array

In [32]:
def compress_zone(np_image_section: np.array, quality: int = 95):
    jpg_img = Image.fromarray(np_image_section)
    buf = BytesIO()
    jpg_img.save(buf, format="JPEG", quality=quality)
    
    return buf.getvalue()

In [33]:
def get_zone_image(np_image: np.array, x: int, y: int, zoom_level: int, w_resolution: int = 1920, h_resolution: int = 1080):
    return compress_zone(select_zone(np_image=np_image, 
                                     x=x, 
                                     y=y, 
                                     zoom_level=zoom_level, 
                                     w_resolution=w_resolution, 
                                     h_resolution=h_resolution))

In [34]:
def save_compression_levels(np_image: np.array, filename: str, exit_w_dim: int = 1920, exit_h_dim: int = 1080):
    w_img = np_image.shape[0]
    h_img = np_image.shape[0]

    for compression_level in range(1, max(w_img/exit_w_dim, h_img/exit_h_dim)):
        x = 0
        while x + exit_w_dim * compression_level <= w_img:
            y = 0
            while y + exit_h_dim * compression_level <= h_img:
                with open(f'{filename}_compression_level_{compression_level}.jpg', 'wb') as f:
                    f.write(get_zone_image(np_image=np_image, x=x, y=y, w_resolution=exit_w_dim, h_resolution=exit_h_dim))